## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [2]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [3]:
# 設定 data_path
dir_data = 'C:\\Users\\USER'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [11]:
app_train.loc[:,'AMT_GOODS_PRICE'].describe()

count    3.072330e+05
mean     5.383962e+05
std      3.694465e+05
min      4.050000e+04
25%      2.385000e+05
50%      4.500000e+05
75%      6.795000e+05
max      4.050000e+06
Name: AMT_GOODS_PRICE, dtype: float64

In [27]:
price_data = app_train[['AMT_GOODS_PRICE']]

In [35]:
price_data["equal_width_price"] = pd.cut(price_data['AMT_GOODS_PRICE'], 4)
print(price_data["equal_width_price"].value_counts())

price_data["equal_freq_price"] = pd.qcut(price_data['AMT_GOODS_PRICE'], 4)
print(price_data["equal_freq_price"].value_counts())

price_data["customized_price_grp"] = pd.cut(price_data['AMT_GOODS_PRICE'], [40499,238500,450000,679500,4050000])
print(price_data["customized_price_grp"].value_counts())

(36490.5, 1042875.0]      274655
(1042875.0, 2045250.0]     31580
(2045250.0, 3047625.0]       968
(3047625.0, 4050000.0]        30
Name: equal_width_price, dtype: int64
(40499.999, 238500.0]    79877
(238500.0, 450000.0]     78102
(679500.0, 4050000.0]    75534
(450000.0, 679500.0]     73720
Name: equal_freq_price, dtype: int64
(40499, 238500]      79877
(238500, 450000]     78102
(679500, 4050000]    75534
(450000, 679500]     73720
Name: customized_price_grp, dtype: int64


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [34]:
price_data.head(10)

,AMT_GOODS_PRICE,equal_width_price,equal_freq_price,customized_age_grp
0,351000.0,"(36490.5, 1042875.0]","(238500.0, 450000.0]","(238500, 450000]"
1,1129500.0,"(1042875.0, 2045250.0]","(679500.0, 4050000.0]","(679500, 4050000]"
2,135000.0,"(36490.5, 1042875.0]","(40499.999, 238500.0]","(40499, 238500]"
3,297000.0,"(36490.5, 1042875.0]","(238500.0, 450000.0]","(238500, 450000]"
4,513000.0,"(36490.5, 1042875.0]","(450000.0, 679500.0]","(450000, 679500]"
5,454500.0,"(36490.5, 1042875.0]","(450000.0, 679500.0]","(450000, 679500]"
6,1395000.0,"(1042875.0, 2045250.0]","(679500.0, 4050000.0]","(679500, 4050000]"
7,1530000.0,"(1042875.0, 2045250.0]","(679500.0, 4050000.0]","(679500, 4050000]"
8,913500.0,"(36490.5, 1042875.0]","(679500.0, 4050000.0]","(679500, 4050000]"
9,405000.0,"(36490.5, 1042875.0]","(238500.0, 450000.0]","(238500, 450000]"


# 等區間的貸款呈現出大部分的人都集中在最小貸款36490.5~ 1042875.0間，而等筆數的貸款似乎就是四分位數的翻版